In [438]:
import pandas as pd 
import numpy as np
import pyecharts
import plotly.express as px # visualization
import pandas_datareader as pdr # access fred
from datetime import date
from fredapi import Fred
import requests
import openpyxl
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta
import re

from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

from pyecharts import options as opts
from pyecharts.charts import Bar, Line

In [276]:
index_data = pd.read_excel('C:/Users/fanjz/Desktop/python/pmi/pmi_level.xlsx')
glance = pd.read_excel('C:/Users/fanjz/Desktop/python/pmi/glancetable.xlsx')
comments = pd.read_excel('C:/Users/fanjz/Desktop/python/pmi/comments.xlsx')
sector_ranking = pd.read_excel('C:/Users/fanjz/Desktop/python/pmi/sector_ranking.xlsx')

In [81]:
url = 'https://www.prnewswire.com/news-releases/manufacturing-pmi-at-49-september-2023-manufacturing-ism-report-on-business-301943535.html#:~:text=The%20report%20was%20issued%20today,47.6%20percent%20recorded%20in%20August.'
response = requests.get(url)

In [305]:
# Extract What Respondents Say

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')   
    section = soup.find('section', class_='release-body container')

    if section:
        main_content = section.find('div', class_='col-lg-10 col-lg-offset-1')

        if main_content:
            ul_element = main_content.find('ul', type='disc')
            if ul_element:
                list_items = ul_element.find_all('li')
                if list_items:
                    # Create a list to store the text of each list item
                    items_list = [item.text.strip() for item in list_items]

                    # Create a DataFrame from the list
                    respondent_comment = pd.DataFrame(items_list, columns=['WHAT RESPONDENTS ARE SAYING'])
        else:
            print("Main content not found within the section.")
    else:
        print("Section with class 'xxxx' not found.")

else:
    print("Failed to retrieve the page")

In [207]:
# Extract monthly index level

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')   
    section = soup.find('section', class_='release-body container')

    if section:
        table_div = section.find('div', class_='table-responsive')

        if table_div:
        # Find the table within the div
            table = table_div.find('table')

            if table:
                # Read the table into a DataFrame
                data_table = pd.read_html(str(table))[0]
            else:
                print("Table not found within the div with class 'table-responsive'")
        else:
            print("Div with class 'table-responsive' not found.")
    else:
        print("Failed to retrieve the page")

C:\Users\fanjz\AppData\Local\Temp\ipykernel_15528\2319469250.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data_table = pd.read_html(str(table))[0]


In [175]:
# Extract Commodity up/down/short supply

commodity_up = soup.select_one('#main > article > section > div > div > p:nth-child(16)')
commodity_down = soup.select_one('#main > article > section > div > div > p:nth-child(17)')
commodity_short = soup.select_one('#main > article > section > div > div > p:nth-child(18)')

commodity_tags = [commodity_up,commodity_down,commodity_short]
commoditiy_direction = []
for i, tag in enumerate(commodity_tags):
    if tag:
        direction = tag.get_text(separator=' ').strip()  # Get all the text within the <p> tag
        commoditiy_direction.append(direction)
    else:
        print(f"The specified <p> tag {i+1} was not found.")

# Create a DataFrame with one column and three rows from the list of contents
commodity_comments = pd.DataFrame(commoditiy_direction, columns=['content'])

In [176]:
substring_patterns = ['Commodities Up in Price','Commodities Down in Price','Commodities in  Short Supply']

def split_based_on_pattern(content, pattern):
    if pattern in content:
        parts = content.split(pattern, 1)
        return [pattern.strip(), parts[1].strip()]
    else:
        return [content.strip(), '']

# Split each row based on the defined patterns
split_data = []
for i, row in commodity_comments.iterrows():
    split_data.append(split_based_on_pattern(row['content'], substring_patterns[i]))

# Create a new DataFrame from the split data
commodity_df = pd.DataFrame(split_data, columns=['pattern', 'rest'])

# Display the split DataFrame
commodity_df

,pattern,rest
0,Commodities Up in Price,Crude Oil (2); Diesel Fuel (2); Electronic Com...
1,Commodities Down in Price,Aluminum (4); Caustic Soda (3); Corrugate Boxe...
2,Commodities in Short Supply,Electrical Components (36); Electrical Transmi...


In [143]:
# Extract Industry Rankings

pmi = soup.select_one('#main > article > section > div > div > p:nth-child(10)')
new_order = soup.select_one('#main > article > section > div > div > p:nth-child(26)')
production = soup.select_one('#main > article > section > div > div > p:nth-child(29)')
employment= soup.select_one('#main > article > section > div > div > p:nth-child(32)')
supplier_deliveries = soup.select_one('#main > article > section > div > div > p:nth-child(35)')
inventories = soup.select_one('#main > article > section > div > div > p:nth-child(38)')
customers_inv = soup.select_one('#main > article > section > div > div > p:nth-child(41)')
prices = soup.select_one('#main > article > section > div > div > p:nth-child(44)')
backlog_orders = soup.select_one('#main > article > section > div > div > p:nth-child(47)')
new_export_orders = soup.select_one('#main > article > section > div > div > p:nth-child(50)')
imports = soup.select_one('#main > article > section > div > div > p:nth-child(53)')

industry_ranking = [pmi,new_order,production,employment,supplier_deliveries,inventories,customers_inv,prices,backlog_orders,new_export_orders,imports]

rankings = []
for i, tag in enumerate(industry_ranking):
    if tag:
        ranking = tag.get_text(separator=' ').strip()  # Get all the text within the <p> tag
        rankings.append(ranking)
    else:
        print(f"The specified <p> tag {i+1} was not found.")

# Create a DataFrame with one column and three rows from the list of contents
ranking_df = pd.DataFrame(rankings, columns=['Ranking'])
ranking_df['Ranking'] = ranking_df['Ranking'].str.strip()


,pattern,rest
0,Commodities Up in Price,Crude Oil (2); Diesel Fuel (2); Electronic Com...
1,Commodities Down in Price,Aluminum (4); Caustic Soda (3); Corrugate Boxe...
2,Commodities in Short Supply,Electrical Components (36); Electrical Transmi...


In [171]:
industry_list = ['PMI','New Order','Production','Employment','Supplier Deliveries','Inventories','Customers_inv','Prices','Backlog Orders','New Export Orders','Imports']
ranking_df['Industry'] = industry_list
col_oder = ['Industry','Ranking']
ranking_df = ranking_df[col_oder]

### Working on Data Clenaing ### 
##### 1. PMI Index Level #####

In [ ]:
# variables = data_table, respondent_comment, commodity_df, ranking_df
# 1. Working on data_table

new_table = data_table.iloc[1:]
header = new_table.iloc[0]
final_table = new_table.iloc[1:]
final_table.columns = header

In [278]:
new_month_level = final_table.iloc[:,1]
new_month_level = new_month_level[:-2]
new_month_level = new_month_level.to_frame().T

In [279]:
latest_date = index_data['Date'].max()
new_date = latest_date + relativedelta(months=1)
new_date_str = new_date.strftime('%Y-%m-%d')
new_month_level["Date"] = new_date
new_month_level = new_month_level[['Date'] + [col for col in new_month_level if col != 'Date']]
new_month_level.columns = index_data.columns

In [285]:
updated_index_data = pd.concat([index_data, new_month_level], ignore_index=True)
updated_index_data.to_excel('C:/Users/fanjz/Desktop/python/pmi/pmi_level.xlsx', index=False)

##### 2. What Respondents Saying #####

In [573]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_colwidth', 80)
respondent_comment

,WHAT RESPONDENTS ARE SAYING
0,"""In the evolving supply chain environment, customers are increasingly taking an active role in initiating new projects, looking for cost reduction opportunities and lead-time mitigation, with a growing emphasis on collaboration. Post-pandemic, customers have learned they need partners to navigate rough waters."" [Computer & Electronic Products]"
1,"""We need to coordinate very closely with suppliers in order to yield a more cost-competitive offer. More back and forth is needed to reach a reasonable total price."" [Chemical Products]"
2,"""Orders and production remain steady, and we are maintaining a healthy backlog. Continued inflation and wage adjustments continue to drive prices up, although we should get some relief from the markets stabilizing."" [Transportation Equipment]"
3,"""Cost increases are now generally isolated to specific commodities rather than blanket increases due to 'inflation.' "" [Food, Beverage & Tobacco Products]"
4,"""Markets remain soft. Our customers have about-right inventory levels, but they paid more due to pandemic cost increases. Everyone is holding off on increasing inventories, hoping they can buy at lower costs."" [Apparel, Leather & Allied Products]"
5,"""Overall, things continue to be very steady: Sales and revenue are as expected, and the supply environment has stabilized greatly versus 2021-22. Some things to watch include the Panama Canal (drought), U.S.-China relations, and the impact the UAW (United Auto Workers) strike could have on suppliers of ours who support automotive production. But overall conditions feel stable."" [Miscellaneous Manufacturing]"
6,"""Cement negotiations have changed, with cement mills no longer offering annual or guaranteed pricing. We now want to contract more as a commodity, leaning toward quarterly, with fluctuating prices yet to be determined."" [Nonmetallic Mineral Products]"
7,"""A recession feels imminent. Money continues to be pushed into the bank markets, driving inflation rates really high. Most plants are buying less material or reducing consumption in the name of sustainability, as well as running at 80 percent of capacity. Prices of some products may increase for the upcoming winter weather."" [Petroleum & Coal Products]"
8,"""Business conditions and market demand remain strong. We are projected to be at capacity in the next 12 months."" [Primary Metals]"
9,"""New business development is coming onboard. However, many forecasts are set for the beginning of 2024. Hiring and retaining quality people is still a struggle."" [Textile Mills]"


In [359]:
updated_comments = respondent_comment['WHAT RESPONDENTS ARE SAYING'].str.split('[', expand=True)
updated_comments.columns = [new_date_str, 'Industry']
updated_comments['Industry'] = updated_comments['Industry'].str.replace(']', '', regex=False)
updated_comments[new_date_str] = updated_comments[new_date_str].str.replace('"', '', regex=False)
updated_comments = updated_comments.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

col_order = ['Industry',new_date_str]
updated_comments = updated_comments[col_order]

In [360]:
industry_col1 = comments['Industry']
industry_col2 = updated_comments['Industry']

merged_df = pd.concat([industry_col1, industry_col2], ignore_index=True)
merged_series = merged_df.squeeze().drop_duplicates()
merge_comment = merged_series.to_frame(name='Industry')

In [361]:
merge_final_df1 = pd.merge(merge_comment, comments, on='Industry', how='left')
final_comment = pd.merge(merge_final_df1, updated_comments, on='Industry', how='left')
final_comment.to_excel('C:/Users/fanjz/Desktop/python/pmi/comments.xlsx', index=False)

##### 3. Commodity Product Pricing #####

In [422]:
commodity_df

commodity_dfs = []

for index, row in commodity_df.iterrows():
    # Create a new DataFrame for the current row
    row_df = pd.DataFrame([row])
    # Append the new DataFrame to the list
    commodity_dfs.append(row_df)
    

In [446]:
commodity_up_df = commodity_dfs[0]
commodity_up_df_split = commodity_up_df['rest'].str.split(';', expand = True).T
commodity_up_header = ['Commodities Up in Price']
commodity_up_df_split.columns = commodity_up_header


In [443]:
# Function to extract the commodity name and any notes
def split_commodity_info(commodity):
    # Remove 'and', '.' and trailing whitespaces
    commodity = re.sub(r'\band\b', '', commodity)
    commodity = commodity.replace('.', '').strip()
    
    # Extract the commodity name and note
    match = re.match(r'(.+?)(?:\s*\((\d+)\)|\s*\*|$)', commodity)
    if match:
        name = match.group(1).strip()
        note = match.group(2) if match.group(2) else '1'  # Replace empty notes with '1'
        return name, note
    return commodity, '1'


In [447]:
# Apply the function to create new columns
commodity_up_df_split[['Commodities Price Up', 'Number of Month']] = commodity_up_df_split['Commodities Up in Price'].apply(split_commodity_info).tolist()

# Drop the original column
commodity_up_df_split.drop(columns=['Commodities Up in Price'], inplace=True)

# Convert 'Note' to integer
commodity_up_df_split['Number of Month'] = commodity_up_df_split['Number of Month'].astype(int)


commodity_down_df

In [449]:
commodity_down_df = commodity_dfs[1]
commodity_down_df_split = commodity_down_df['rest'].str.split(';', expand = True).T
commodity_down_header = ['Commodities Down in Price']
commodity_down_df_split.columns = commodity_down_header

In [452]:
# Apply the function to create new columns
commodity_down_df_split[['Commodities Price Down', 'Number of Month']] = commodity_down_df_split['Commodities Down in Price'].apply(split_commodity_info).tolist()

# Drop the original column
commodity_down_df_split.drop(columns=['Commodities Down in Price'], inplace=True)

# Convert 'Note' to integer
commodity_down_df_split['Number of Month'] = commodity_down_df_split['Number of Month'].astype(int)

In [453]:
commodity_down_df_split

,Commodities Price Down,Number of Month
0,Aluminum,4
1,Caustic Soda,3
2,Corrugate Boxes,2
3,Ocean Freight Rates,1
4,Plastic Resins,1
5,Polypropylene,5
6,Steel,1
7,Steel — Hot Rolled,5
8,Steel Products,4


Commodity in short supply

In [456]:
commodity_short_df = commodity_dfs[2]
commodity_short_df_split = commodity_short_df['rest'].str.split(';', expand = True).T
commodity_short_header = ['Commodities in Short Supply']
commodity_short_df_split.columns = commodity_short_header

In [457]:
# Apply the function to create new columns
commodity_short_df_split[['Commodities: Short Supply', 'Number of Month']] = commodity_short_df_split['Commodities in Short Supply'].apply(split_commodity_info).tolist()

# Drop the original column
commodity_short_df_split.drop(columns=['Commodities in Short Supply'], inplace=True)

# Convert 'Note' to integer
commodity_short_df_split['Number of Month'] = commodity_short_df_split['Number of Month'].astype(int)

In [ ]:
commodity_short_df_split.to_excel('C:/Users/fanjz/Desktop/python/pmi/commmodity_short.xlsx', index=False)
commodity_up_df_split.to_excel('C:/Users/fanjz/Desktop/python/pmi/commmodity_up.xlsx', index=False)
commodity_down_df_split.to_excel('C:/Users/fanjz/Desktop/python/pmi/commmodity_down.xlsx', index=False)

##### 4. Industry Ranking #####

In [532]:
# Further split col that contains 'is'
def split_col(row):
    if 'is' in row[0]:
        parts = row[0].split('is', 1)
        return pd.Series(parts + row[1:].tolist())
    else:
        return row

# Further split col1 by ';'
def split_col1(row):
    if pd.notnull(row[1]) and ';' in row[1]:
        parts = row[1].split(';')
        return pd.Series([row[0]] + parts + row[2:].tolist())
    else:
        return row

In [4]:
ranking_df_split = ranking_df['Ranking'].str.split('.', expand = True)
ranking_df_split.columns = ['Industry Up','Industry Down','No Change','None']

NameError: name 'ranking_df' is not defined

In [3]:
ranking_up = pd.concat([ranking_df['Industry'], ranking_df_split['Industry Up']], axis=1)
ranking_down = pd.concat([ranking_df['Industry'], ranking_df_split['Industry Down']], axis=1)

NameError: name 'pd' is not defined

In [1]:
ranking_up = ranking_up['Industry Up'].str.split(':', expand=True)
ranking_down = ranking_down['Industry Down'].str.split('are:', expand=True)

NameError: name 'ranking_up' is not defined

In [533]:
ranking_up = ranking_up.apply(split_col, axis=1)
ranking_up = ranking_up.apply(split_col1, axis=1)
ranking_up = ranking_up.applymap(lambda x: x.replace('and', '').strip() if isinstance(x, str) else x)

In [578]:
ranking_up = pd.concat([ranking_df['Industry'], ranking_up.iloc[:, 1:]], axis=1)

In [579]:
stack_df = ranking_up.melt(id_vars=["Industry"])

In [580]:
melted_df = stack_df.sort_values(['Industry','variable'])

In [581]:
value_order = ranking_df['Industry'].to_list()
melted_df['Industry'] = pd.Categorical(melted_df['Industry'], categories=value_order, ordered=True)
melted_df = melted_df.sort_values(['Industry', 'variable']).reset_index(drop=True)
melted_df.columns = ['Index','Ranking','Industry']


In [583]:
melted_df = melted_df.dropna(subset=['Industry'])

In [584]:
melted_df

,Index,Ranking,Industry
0,PMI,1,Nonmetallic Mineral Products
1,PMI,2,"Food, Beverage & Tobacco Products"
2,PMI,3,Textile Mills
3,PMI,4,Primary Metals
4,PMI,5,Petroleum & Coal Products
11,New Order,1,Miscellaneous Manufacturing
12,New Order,2,"Food, Beverage & Tobacco Products"
13,New Order,3,Primary Metals
14,New Order,4,Fabricated Metal Products
15,New Order,5,Nonmetallic Mineral Products


In [591]:
ranking_down = ranking_down.apply(split_col, axis=1)
ranking_down = ranking_down.apply(split_col1, axis=1)
ranking_down = ranking_down.applymap(lambda x: x.replace('and', '').strip() if isinstance(x, str) else x)

C:\Users\fanjz\AppData\Local\Temp\ipykernel_15528\1383675453.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ranking_down = ranking_down.applymap(lambda x: x.replace('and', '').strip() if isinstance(x, str) else x)


In [592]:
ranking_down

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,The 11 industries reporting contraction in September — in the following order — are,Printing & Related Support Activities,Furniture & Related Products,Plastics & Rubber Products,Paper Products,Fabricated Metal Products,Wood Products,Computer & Electronic Products,Machinery,"Electrical Equipment, Appliances & Components",Chemical Products,Transportation Equipment,NaN,NaN
1,"Eight industries reported a decline in new orders in September, in the following order",Printing & Related Support Activities,Furniture & Related Products,"Electrical Equipment, Appliances & Components",Petroleum & Coal Products,Machinery,Transportation Equipment,Computer & Electronic Products,Chemical Products,NaN,NaN,NaN,NaN,NaN
2,The six industries reporting a decrease in production in September — in the following order — are,Printing & Related Support Activities,Furniture & Related Products,Wood Products,Plastics & Rubber Products,Paper Products,Chemical Products,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"The eight industries reporting a decrease in employment in September, in the following order, are",Printing & Related Support Activities,Furniture & Related Products,Textile Mills,Computer & Electronic Products,Plastics & Rubber Products,Miscellaneous Manufacturing,Fabricated Metal Products,"Electrical Equipment, Appliances & Components",NaN,NaN,NaN,NaN,NaN
4,The nine industries reporting faster supplier deliveries in September as compared to August — in the following order — are,Plastics & Rubber Products,Primary Metals,Fabricated Metal Products,Machinery,Miscellaneous Manufacturing,"Electrical Equipment, Appliances & Components",Computer & Electronic Products,"Food, Beverage & Tobacco Products",Chemical Products,NaN,NaN,NaN,NaN
5,The 11 industries reporting contracting inventories in September — in the following order — are,Printing & Related Support Activities,Paper Products,Furniture & Related Products,Wood Products,Plastics & Rubber Products,Fabricated Metal Products,Transportation Equipment,Miscellaneous Manufacturing,Primary Metals,Computer & Electronic Products,Machinery,NaN,NaN
6,The 10 industries reporting customers' inventories as too low in September — in the following order — are,Wood Products,Textile Mills,Petroleum & Coal Products,Nonmetallic Mineral Products,Furniture & Related Products,Primary Metals,Machinery,"Food, Beverage & Tobacco Products",Transportation Equipment,Chemical Products,NaN,NaN,NaN
7,The 12 industries reporting paying decreased prices for raw materials in September — in the following order — are,Fabricated Metal Products,Paper Products,Furniture & Related Products,Textile Mills,"Electrical Equipment, Appliances & Components","Food, Beverage & Tobacco Products",Transportation Equipment,Machinery,Plastics & Rubber Products,Miscellaneous Manufacturing,Primary Metals,Chemical Products,NaN
8,The 11 industries reporting lower backlogs in September — in the following order — are,Paper Products,Nonmetallic Mineral Products,Plastics & Rubber Products,Computer & Electronic Products,"Electrical Equipment, Appliances & Components",Machinery,Furniture & Related Products,Transportation Equipment,"Food, Beverage & Tobacco Products",Miscellaneous Manufacturing,Fabricated Metal Products,NaN,NaN
9,The six industries reporting a decrease in new export orders in September — in the following order — are,Plastics & Rubber Products,Computer & Electronic Products,Miscellaneous Manufacturing,Fabricated Metal Products,"Electrical Equipment, Appliances & Components",Machinery,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [593]:
ranking_df_split

,Industry Up,Industry Down,No Change,None
0,"The five manufacturing industries that reported growth in September are: Nonmetallic Mineral Products; Food, Beverage & Tobacco Products; Textile Mills; Primary Metals; and Petroleum & Coal Products","The 11 industries reporting contraction in September — in the following order — are: Printing & Related Support Activities; Furniture & Related Products; Plastics & Rubber Products; Paper Products; Fabricated Metal Products; Wood Products; Computer & Electronic Products; Machinery; Electrical Equipment, Appliances & Components; Chemical Products; and Transportation Equipment",,None
1,"The five manufacturing industries that reported growth in new orders in September are: Miscellaneous Manufacturing; Food, Beverage & Tobacco Products; Primary Metals; Fabricated Metal Products; and Nonmetallic Mineral Products","Eight industries reported a decline in new orders in September, in the following order: Printing & Related Support Activities; Furniture & Related Products; Electrical Equipment, Appliances & Components; Petroleum & Coal Products; Machinery; Transportation Equipment; Computer & Electronic Products; and Chemical Products",,None
2,"The 10 industries reporting growth in production during the month of September — in the following order — are: Primary Metals; Textile Mills; Miscellaneous Manufacturing; Food, Beverage & Tobacco Products; Nonmetallic Mineral Products; Electrical Equipment, Appliances & Components; Machinery; Transportation Equipment; Fabricated Metal Products; and Computer & Electronic Products",The six industries reporting a decrease in production in September — in the following order — are: Printing & Related Support Activities; Furniture & Related Products; Wood Products; Plastics & Rubber Products; Paper Products; and Chemical Products,,None
3,"Of 18 manufacturing industries, seven reported employment growth in September in the following order: Petroleum & Coal Products; Food, Beverage & Tobacco Products; Primary Metals; Transportation Equipment; Chemical Products; Machinery; and Nonmetallic Mineral Products","The eight industries reporting a decrease in employment in September, in the following order, are: Printing & Related Support Activities; Furniture & Related Products; Textile Mills; Computer & Electronic Products; Plastics & Rubber Products; Miscellaneous Manufacturing; Fabricated Metal Products; and Electrical Equipment, Appliances & Components",,None
4,The five manufacturing industries reporting slower supplier deliveries in September are: Wood Products; Textile Mills; Paper Products; Nonmetallic Mineral Products; and Transportation Equipment,"The nine industries reporting faster supplier deliveries in September as compared to August — in the following order — are: Plastics & Rubber Products; Primary Metals; Fabricated Metal Products; Machinery; Miscellaneous Manufacturing; Electrical Equipment, Appliances & Components; Computer & Electronic Products; Food, Beverage & Tobacco Products; and Chemical Products",,None
5,"Of 18 manufacturing industries, the two reporting higher inventories in September are: Nonmetallic Mineral Products; and Electrical Equipment, Appliances & Components",The 11 industries reporting contracting inventories in September — in the following order — are: Printing & Related Support Activities; Paper Products; Furniture & Related Products; Wood Products; Plastics & Rubber Products; Fabricated Metal Products; Transportation Equipment; Miscellaneous Manufacturing; Primary Metals; Computer & Electronic Products; and Machinery,,None
6,The four industries reporting customers' inventories as too high in September are: Plastics & Rubber Products; Computer & Electronic Products; Miscellaneous Manufacturing; and Fabricated Metal Products,"The 10 industries reporting customers' inventories as too low in September — in the following order — are: Wood Products; Textile Mills; Petroleum & Coal Products; Nonmetallic Mineral Products; Furniture & Relat